In [16]:
import pandas as pd
import joblib

# Cargar el modelo y los encoders
model = joblib.load('car_price_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Cargar el dataset original para obtener la relación entre marcas y modelos
df = pd.read_csv('car_price_dataset.csv')

# Obtener valores válidos de los encoders
marcas_validas = list(label_encoders['Brand'].classes_)
combustibles_validos = list(label_encoders['Fuel_Type'].classes_)
transmisiones_validas = list(label_encoders['Transmission'].classes_)

# Función para solicitar entrada validada
def solicitar_input(mensaje, tipo, valid_values=None):
    while True:
        valor = input(mensaje).strip()
        if tipo == int:
            if not valor.isdigit():
                print("Error: Debes ingresar un número entero.")
                continue
            return int(valor)
        
        elif tipo == float:
            try:
                return float(valor)
            except ValueError:
                print("Error: Debes ingresar un número válido con decimales (ej: 1.8).")
                continue

        elif tipo == str:
            if valid_values and valor not in valid_values:
                print(f"Error: El valor '{valor}' no es válido. Opciones disponibles: {', '.join(valid_values)}")
                continue
            return valor

# Pedir la marca del coche
marca_seleccionada = solicitar_input(f"Introduce la marca del coche ({', '.join(marcas_validas)}): ", str, marcas_validas)

# Filtrar modelos que pertenecen a la marca seleccionada
modelos_validos = df[df['Brand'] == marca_seleccionada]['Model'].unique().tolist()

# Pedir el modelo dentro de los disponibles para la marca seleccionada
modelo_seleccionado = solicitar_input(f"Introduce el modelo del coche ({', '.join(modelos_validos)}): ", str, modelos_validos)

# Seguir pidiendo el resto de datos
nuevo_coche = {
    'Brand': marca_seleccionada,
    'Model': modelo_seleccionado,
    'Year': solicitar_input("Introduce el año del coche: ", int),
    'Engine_Size': solicitar_input("Introduce el tamaño del motor (en litros): ", float),
    'Fuel_Type': solicitar_input(f"Introduce el tipo de combustible ({', '.join(combustibles_validos)}): ", str, combustibles_validos),
    'Transmission': solicitar_input(f"Introduce el tipo de transmisión ({', '.join(transmisiones_validas)}): ", str, transmisiones_validas),
    'Mileage': solicitar_input("Introduce el kilometraje: ", int),
    'Doors': solicitar_input("Introduce el número de puertas: ", int),
    'Owner_Count': solicitar_input("Introduce el número de dueños anteriores: ", int)
}

# Convertir a DataFrame
nuevo_coche_df = pd.DataFrame([nuevo_coche])

# Convertir las variables categóricas a su codificación numérica
categorical_columns = ['Brand', 'Model', 'Fuel_Type', 'Transmission']
for column in categorical_columns:
    nuevo_coche_df[column] = label_encoders[column].transform(nuevo_coche_df[column])

# Asegurar que las columnas están en el mismo orden que el modelo espera
nuevo_coche_df = nuevo_coche_df[model.feature_names_in_]

# Realizar la predicción
prediccion = model.predict(nuevo_coche_df)

print(f'El precio estimado del coche es: {prediccion[0]:,.2f} $')

Introduce la marca del coche (Audi, BMW, Chevrolet, Ford, Honda, Hyundai, Kia, Mercedes, Toyota, Volkswagen):  Kia
Introduce el modelo del coche (Rio, Sportage, Optima):  Sportage
Introduce el año del coche:  2014
Introduce el tamaño del motor (en litros):  2.6
Introduce el tipo de combustible (Diesel, Electric, Hybrid, Petrol):  Hybrid
Introduce el tipo de transmisión (Automatic, Manual, Semi-Automatic):  Manual
Introduce el kilometraje:  98700
Introduce el número de puertas:  3
Introduce el número de dueños anteriores:  4


El precio estimado del coche es: 10,154.83 $
